# Explosions and extinctions

Major changes in a domain...

<p class="alert alert-warning">Work in progress – this notebook isn't finished yet. Check back later for more...<p>

In [89]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from tqdm.auto import tqdm
import pandas as pd
import requests_cache
import time
import altair as alt

s = requests.Session()
retries = Retry(total=10, backoff_factor=1, status_forcelist=[ 502, 503, 504 ])
s.mount('https://', HTTPAdapter(max_retries=retries))
s.mount('http://', HTTPAdapter(max_retries=retries))

alt.data_transformers.enable('json', urlpath='files')

DataTransformerRegistry.enable('json')

In [90]:
params = {
    'url': 'gov.au',
    'matchType': 'domain',
    'filter': ['statuscode:200', 'mimetype:text/html'],
    'collapse': 'urlkey',
    'fl': 'urlkey,mimetype,statuscode'
}

def get_total_pages(params):
    these_params = params.copy()
    these_params['showNumPages'] = 'true'
    response = s.get('http://web.archive.org/cdx/search/cdx', params=these_params, headers={'User-Agent': ''})
    return int(response.text)

def query_cdx(url, **kwargs):
    results = []
    page = 0
    params = kwargs
    params['url'] = url
    params['output'] = 'json'
    total_pages = get_total_pages(params)
    print(total_pages)
    with tqdm(total=total_pages-page) as pbar:
        while page < total_pages:
            params['page'] = page
            response = requests.get('http://web.archive.org/cdx/search/cdx', params=params, headers={'User-Agent': ''})
            print(response.url)
            response.raise_for_status()
            if page == 0:
                results = response.json()
            else:
                results += response.json()[1:]
            page += 1
            pbar.update(1)
            time.sleep(0.2)
    return results

In [91]:
params = {
    'url': 'nla.gov.au',
    'output': 'json'
}
get_total_pages(params)

1

In [ ]:
results = query_cdx('nla.gov.au/*', filter=['statuscode:404'])

In [64]:
df = pd.DataFrame(results[1:], columns=results[0])

In [32]:
df.head()

,urlkey,timestamp,original,mimetype,statuscode,digest,length
0,"au,gov,nla)/!==a.direction?&&&&!1:a.atend&0===...",20130113015219,http://www.nla.gov.au/!==a.direction?!1:a.atEn...,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5390
1,"au,gov,nla)/""'+",20031023020002,http://www.nla.gov.au:80/%22'+,text/html,404,3663VAIO4OLGLCU6OIQ2JBI7PCPJMAJS,1185
2,"au,gov,nla)/""'+",20031209013805,"http://www.nla.gov.au:80/""'+",text/html,404,V2IHKASEC74N55K2ZC5DEMF2JD45PZWU,3354
3,"au,gov,nla)/""'+",20031210011400,"http://www.nla.gov.au:80/""'+",text/html,404,V2IHKASEC74N55K2ZC5DEMF2JD45PZWU,3354
4,"au,gov,nla)/""'+",20040221225936,"http://www.nla.gov.au:80/""'+",text/html,404,TNIQ75VBIWHP64Q3CY4WKYTEBV4H4536,3374


In [33]:
df.shape

(88801, 7)

In [67]:
# Convert the timestamp string into a datetime object
df['date'] = pd.to_datetime(df['timestamp'])
df.sort_values(by='date', inplace=True, ignore_index=True)

# df.drop_duplicates(subset=['urlkey'], keep='first', inplace=True)

In [68]:
df.shape

(27974, 8)

In [69]:
alt.Chart(df).mark_bar().encode(
    x = 'yearmonth(date):T',
    y = 'count():Q'
).properties(width=700)

alt.Chart(...)

In [70]:
df.loc[(df['date'] >= '2012-01-01') & (df['date'] <= '2012-12-31')]

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date
52682,"au,gov,nla)/find/people.html",20120107041711,http://www.nla.gov.au:80/find/people.html,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5258,2012-01-07 04:17:11
52685,"au,gov,nla)/librariesaustralia/april-2011",20120107043903,http://www.nla.gov.au:80/librariesaustralia/ap...,text/html,404,2575F7F3MVVP5RGJN3RCIUY6LHYSVIVJ,2623,2012-01-07 04:39:03
52697,"au,gov,nla)/nla.cs-pa-http:/stonlib.stonningto...",20120108011326,http://nla.gov.au:80/nla.cs-pa-http://stonlib....,text/html,404,X4CCVFN3FFTKYFALG3XY6T5VRTS5EGKA,1427,2012-01-08 01:13:26
52701,"au,gov,nla)/nla.cs-pa-http:/emuseum.anmm.gov.a...",20120110055950,http://nla.gov.au:80/nla.cs-pa-http://emuseum....,text/html,404,X4CCVFN3FFTKYFALG3XY6T5VRTS5EGKA,1403,2012-01-10 05:59:50
52702,"au,gov,nla)/epubs/waltzingmatilda/bibliography...",20120111011220,http://www.nla.gov.au:80/epubs/waltzingmatilda...,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5275,2012-01-11 01:12:20
...,...,...,...,...,...,...,...,...
63113,"au,gov,nla)/preserve/paper/lbrandis.html",20121204015932,http://www.nla.gov.au:80/preserve/paper/lbrand...,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5209,2012-12-04 01:59:32
63204,"au,gov,nla)/news/newsevents.html",20121208053514,http://www.nla.gov.au:80/news/newsevents.html,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5206,2012-12-08 05:35:14
63306,"au,gov,nla)/news/story.php?id=409",20121225233724,http://www.nla.gov.au:80/news/story.php?id=409,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5214,2012-12-25 23:37:24
63308,"au,gov,nla)/www.nla.gov.au/about",20121227103402,http://www.nla.gov.au:80/www.nla.gov.au/about,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5202,2012-12-27 10:34:02


In [71]:
alt.Chart(df.loc[(df['date'] >= '2012-01-01') & (df['date'] <= '2012-12-31')]).mark_bar().encode(
    x = 'month(date):T',
    y = 'count():Q'
)

alt.Chart(...)

In [72]:
from urllib.parse import urlparse
import re

paths = []
for row in df.loc[(df['date'] >= '2012-01-01') & (df['date'] <= '2012-12-31')].itertuples():
    parsed = urlparse(row.original)
    paths += [w for w in re.split(r'\W+', parsed[2]) if w != '']

In [73]:
from collections import Counter
c = Counter(paths)
c.most_common(50)

[('css', 2464),
 ('sites', 1834),
 ('all', 1825),
 ('modules', 1814),
 ('js', 1598),
 ('outgoing', 1318),
 ('nla', 1288),
 ('au', 1231),
 ('gov', 1167),
 ('librariesaustralia', 1098),
 ('www', 1008),
 ('themes', 712),
 ('nla7', 631),
 ('system', 627),
 ('jcarousel', 575),
 ('html', 568),
 ('2011', 496),
 ('media', 429),
 ('releases', 367),
 ('views_slideshow', 322),
 ('views_slideshow_xtra', 316),
 ('jquery', 314),
 ('panels', 312),
 ('images', 294),
 ('node', 270),
 ('pdf', 256),
 ('files', 247),
 ('misc', 244),
 ('views', 236),
 ('date', 234),
 ('com', 228),
 ('services', 224),
 ('blogs', 208),
 ('exhibitions', 206),
 ('2012', 201),
 ('collections', 190),
 ('user', 178),
 ('default', 177),
 ('theme', 164),
 ('1', 160),
 ('library', 159),
 ('documents', 158),
 ('field', 158),
 ('ctools', 156),
 ('aggregator', 156),
 ('11', 153),
 ('the', 148),
 ('mp3', 147),
 ('jpg', 140),
 ('05', 139)]

## Mimetypes and formats

In [74]:
df['extension'] = df['original'].str.extract(r'\.(\w+)$')

In [75]:
df['extension'].value_counts()[:25]

html             5673
js               2703
css              2334
au               1619
jpg              1420
pic              1405
gif               836
htm               494
pdf               388
png               225
sid               149
txt               127
mp3               108
ppt               100
swf                54
doc                53
0                  43
JPG                42
mso                33
com                26
XMLHTTP            25
specialSubmit      20
rss                18
jcarousel          18
Image6             15
Name: extension, dtype: int64

In [76]:
df_formats = df.loc[df['extension'].isin(['html', 'css', 'jpg', 'gif', 'htm', 'pdf', 'png', 'mp3', 'txt', 'ppt', 'swf', 'doc'])]

In [77]:
alt.Chart(df_formats).mark_bar().encode(
    x = 'year(date):T',
    y = 'count():Q',
    color = 'extension'
).properties(width=700)

alt.Chart(...)

In [79]:
df.loc[(df['date'] >= '2012-01-01') & (df['date'] <= '2012-12-31') & (df['extension'] == 'pdf')]

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date,extension
52742,"au,gov,nla)/asian/pub/ial/documents/ialcons316...",20120117095116,http://www.nla.gov.au/asian/pub/ial/documents/...,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5436,2012-01-17 09:51:16,pdf
52743,"au,gov,nla)/ndp/project_details/documents/andp...",20120117105622,http://www.nla.gov.au/ndp/project_details/docu...,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5321,2012-01-17 10:56:22,pdf
53000,"au,gov,nla)/asian/pub/ial/032004.pdf",20120127111718,http://www.nla.gov.au/asian/pub/ial/032004.pdf,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5423,2012-01-27 11:17:18,pdf
53001,"au,gov,nla)/asian/pub/ial/042004.pdf",20120127112350,http://www.nla.gov.au/asian/pub/ial/042004.pdf,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5420,2012-01-27 11:23:50,pdf
53002,"au,gov,nla)/asian/pub/ial/032000.pdf",20120127112807,http://www.nla.gov.au/asian/pub/ial/032000.pdf,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5423,2012-01-27 11:28:07,pdf
...,...,...,...,...,...,...,...,...,...
62026,"au,gov,nla)/initiatives/meetings/sitelicense/d...",20120923103107,http://www.nla.gov.au/initiatives/meetings/sit...,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5217,2012-09-23 10:31:07,pdf
62031,"au,gov,nla)/policy/annrep10/nla-ar10-3-report-...",20120924082146,http://www.nla.gov.au/policy/annrep10/NLA-AR10...,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5378,2012-09-24 08:21:46,pdf
62090,"au,gov,nla)/librariesaustralia/news/documents/...",20120930193540,http://www.nla.gov.au/librariesaustralia/news/...,text/html,404,6O2H6GT5ZZHX2M3NYO6Q3FTNU22JAMUE,2902,2012-09-30 19:35:40,pdf
62844,"au,gov,nla)/asian/pub/ial/documents/ialcons311...",20121102190355,http://www.nla.gov.au/asian/pub/ial/documents/...,text/html,404,HGOGRUY4UG5OD6CV5M4PLTFKE7RDSZJS,5379,2012-11-02 19:03:55,pdf


In [78]:
df.loc[df['original'].str.contains('/dsp/rft/model.pdf')]

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date,extension
49115,"au,gov,nla)/dsp/rft/model.pdf",20110322170427,http://www.nla.gov.au/dsp/rft/model.pdf,text/html,404,KCFR556KFQBODCRCTGPXGRHPB52EEUJV,5437,2011-03-22 17:04:27,pdf


In [80]:
df.loc[(df['date'] >= '2000-01-01') & (df['date'] <= '2000-12-31') & (df['extension'] == 'gif')]

,urlkey,timestamp,original,mimetype,statuscode,digest,length,date,extension
1608,"au,gov,nla)/2/pubs/gw/13/g13a.gif",20000108052608,http://www.nla.gov.au:80/2/pubs/gw/13/g13a.gif,text/html,404,DQUCVFRORM4TIETSH2PAWQBAIE57BFTQ,858,2000-01-08 05:26:08,gif
1610,"au,gov,nla)/2/pubs/gw/13/g13b.gif",20000108070547,http://www.nla.gov.au:80/2/pubs/gw/13/g13b.gif,text/html,404,DQUCVFRORM4TIETSH2PAWQBAIE57BFTQ,860,2000-01-08 07:05:47,gif
1612,"au,gov,nla)/2/pubs/gw/13/g13c.gif",20000108084252,http://www.nla.gov.au:80/2/pubs/gw/13/g13c.gif,text/html,404,DQUCVFRORM4TIETSH2PAWQBAIE57BFTQ,859,2000-01-08 08:42:52,gif
1614,"au,gov,nla)/2/pubs/gw/13/g13d.gif",20000108102056,http://www.nla.gov.au:80/2/pubs/gw/13/g13d.gif,text/html,404,DQUCVFRORM4TIETSH2PAWQBAIE57BFTQ,860,2000-01-08 10:20:56,gif
1616,"au,gov,nla)/2/pubs/gw/13/g13e.gif",20000108115840,http://www.nla.gov.au:80/2/pubs/gw/13/g13e.gif,text/html,404,DQUCVFRORM4TIETSH2PAWQBAIE57BFTQ,861,2000-01-08 11:58:40,gif
...,...,...,...,...,...,...,...,...,...
5182,"au,gov,nla)/ohs/chk3t.gif",20001011213536,http://www.nla.gov.au:80/ohs/chk3t.gif,text/html,404,JVFR4M27H2ID63Z5XNXKS5NTV5KFQABD,869,2000-10-11 21:35:36,gif
5192,"au,gov,nla)/pub/cards/foster/nl98003s.gif",20001012092328,http://www.nla.gov.au:80/pub/cards/foster/nl98...,text/html,404,D7JLWO3HYAZMUJAE3DOZWLS2PJP26ARU,1741,2000-10-12 09:23:28,gif
5197,"au,gov,nla)/pub/cards/foster/nl98002s.gif",20001012162212,http://www.nla.gov.au:80/pub/cards/foster/nl98...,text/html,404,D7JLWO3HYAZMUJAE3DOZWLS2PJP26ARU,1740,2000-10-12 16:22:12,gif
5199,"au,gov,nla)/ohs/chk5t.gif",20001012173247,http://www.nla.gov.au:80/ohs/chk5t.gif,text/html,404,JVFR4M27H2ID63Z5XNXKS5NTV5KFQABD,866,2000-10-12 17:32:47,gif


In [ ]:
df.loc[df['extension'] == 'pdf']['urlkey'].to_list()